In [1]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_dccrime_dbname, remote_dccrime_dbuser, remote_dccrime_dbpwd
#from config import local_dccrime_dbname, local_dccrime_dbuser, local_dccrime_dbpwd

# Import Pandas
import pandas as pd

In [2]:
# Create a remote database engine connection
# AWS Database Connection
engine = create_engine(f"mysql://{remote_dccrime_dbuser}:{remote_dccrime_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_dccrime_dbname}")
conn = engine.connect()



In [3]:
#crime_sexabuse = pd.read_sql("SELECT OFFENSE, BLOCK_GROUP, COUNT(*) from crime_incidents_all WHERE OFFENSE IN ('SEX ABUSE') GROUP BY OFFENSE, BLOCK_GROUP", conn)
#print(len(crime_sexabuse))
#crime_sexabuse.head(50)

violent_crime = pd.read_sql("SELECT OFFENSE, BLOCK_GROUP, COUNT(*) from crime_incidents_all WHERE OFFENSE IN ('ASSAULT W/DANGEROUS WEAPON', 'HOMICIDE', 'ROBBERY', 'SEX ABUSE') GROUP BY OFFENSE, BLOCK_GROUP", conn)
print(len(violent_crime))
violent_crime.head(50)

1432


,OFFENSE,BLOCK_GROUP,COUNT(*)
0,ASSAULT W/DANGEROUS WEAPON,,9
1,ASSAULT W/DANGEROUS WEAPON,000100 1,3
2,ASSAULT W/DANGEROUS WEAPON,000100 2,3
3,ASSAULT W/DANGEROUS WEAPON,000100 3,7
4,ASSAULT W/DANGEROUS WEAPON,000100 4,31
5,ASSAULT W/DANGEROUS WEAPON,000201 1,1
6,ASSAULT W/DANGEROUS WEAPON,000202 1,7
7,ASSAULT W/DANGEROUS WEAPON,000202 2,1
8,ASSAULT W/DANGEROUS WEAPON,000202 3,19
9,ASSAULT W/DANGEROUS WEAPON,000202 4,24


In [4]:
violent_crime_df = violent_crime.rename(columns={"OFFENSE":"Offense", "BLOCK_GROUP":"Block Group", "COUNT(*)": "Count"})
violent_crime_df['Block Group'] = violent_crime_df['Block Group'].str.replace(" ","")
violent_crime_df.head()

,Offense,Block Group,Count
0,ASSAULT W/DANGEROUS WEAPON,,9
1,ASSAULT W/DANGEROUS WEAPON,0001001,3
2,ASSAULT W/DANGEROUS WEAPON,0001002,3
3,ASSAULT W/DANGEROUS WEAPON,0001003,7
4,ASSAULT W/DANGEROUS WEAPON,0001004,31


In [5]:
grouped_violent = violent_crime_df.groupby(['Block Group', 'Offense'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
#print(grouped_usa_df)

# In order to be visualized, a data function must be used...
grouped_violent_df = grouped_violent.sum()

In [6]:
grouped_violent_df2 = grouped_violent_df.unstack()
grouped_violent_df2.head()

#renamed_df = grouped_violent_df2.rename(columns={"ASSAULT W/DANGEROUS WEAPON":"ADW", "HOMICIDE":"Homicide", "ROBBERY": "Robbery", "SEX ABUSE": "Sex Abuse"})
#renamed_df.head()

Count                           
Offense     ASSAULT W/DANGEROUS WEAPON HOMICIDE ROBBERY SEX ABUSE
Block Group                                                      
                                   9.0      2.0     4.0       2.0
0001001                            3.0      NaN     2.0       1.0
0001002                            3.0      NaN    10.0       2.0
0001003                            7.0      NaN     8.0       2.0
0001004                           31.0      NaN    36.0       7.0

In [18]:
#renamed_df.index = renamed_df.index.droplevel(0)
ADW_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["ASSAULT W/DANGEROUS WEAPON"]
homicide_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["HOMICIDE"]
robbery_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["ROBBERY"]
sexabuse_sum = grouped_violent_df2.groupby(["Block Group"]).sum()["Count"]["SEX ABUSE"]

crime_totals = pd.DataFrame({"ADW": ADW_sum,
                                   "Homicide": homicide_sum,
                                   "Robbery": robbery_sum,
                                   "Sex Abuse": sexabuse_sum})


crime_totals.drop(crime_totals.index[0], inplace=True)

crime_totals['BG'] = crime_totals.index
crime_totals.head()
#crime_totals_df = crime_totals


#crime_totals.index.values
#ADW_sum
#crime_totals.index.tolist()

,ADW,Homicide,Robbery,Sex Abuse,BG
Block Group,,,,,
0001001,3.0,0.0,2.0,1.0,0001001
0001002,3.0,0.0,10.0,2.0,0001002
0001003,7.0,0.0,8.0,2.0,0001003
0001004,31.0,0.0,36.0,7.0,0001004
0002011,1.0,0.0,5.0,4.0,0002011


In [ ]:
#THE ABOVE WORKS! 

In [19]:
engine = create_engine(f"mysql://{remote_dccrime_dbuser}:{remote_dccrime_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_dccrime_dbname}")
conn = engine.connect()

#socioecon_data_df.to_sql('socioecon_data', conn)
crime_totals.to_sql(name='crime_totals', if_exists='replace', con=conn, chunksize=1000, index=False)